# Week 01 — Distribution & KPI Sense (분포/대표값/이상치)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 평균 매출이 늘었는데 체감이 없다면 분포(상위 소수/이상치) 때문인가?
- 국가/채널/세그먼트별로 Mean vs Median 차이가 큰 곳은 어디인가?
- KPI를 Median/P75로 재정의하면 인사이트가 달라지는가?

## 2) Why this matters (Theory)
- Skewed distribution에서 Mean은 상위 소수에 과도하게 영향을 받음
- Median/P75는 'typical customer' 혹은 '상위 안정 구간'을 더 잘 설명
- 이상치는 오류가 아니라 '비즈니스 이벤트'일 수 있음

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Revenue distribution (clipped) & summary stats
✅ 매출 분포를 보고 **Mean/Median/P75/P90**를 비교하세요.

In [ ]:
x = df['revenue'].fillna(0)
print("Mean:", x.mean())
print("Median:", x.median())
print("P75:", x.quantile(0.75), "P90:", x.quantile(0.90), "P99:", x.quantile(0.99))

plt.figure()
plt.hist(x.clip(upper=x.quantile(0.99)), bins=60)
plt.title("Revenue distribution (clipped at 99th pct)")
plt.xlabel("revenue")
plt.ylabel("count")
plt.show()

In [ ]:
one_sentence_insight('Mean과 Median이 벌어지는 이유(상위 소수 vs 일반 고객)를 한 문장으로 정리')

### 5.2 Segment matrix (country × channel) with Mean vs Median
✅ Mean과 Median이 크게 다른 조합을 찾아 원인을 추정하세요.

In [ ]:
g = (df.assign(revenue=df['revenue'].fillna(0))
       .groupby(['country','channel'])
       .agg(n=('revenue','size'),
            revenue_mean=('revenue','mean'),
            revenue_median=('revenue','median'),
            revenue_p90=('revenue', lambda s: s.quantile(0.90)),
            conv_rate=('conversions','mean'),
            spend_mean=('marketing_spend','mean'))
       .reset_index())

g['mean_minus_median'] = g['revenue_mean'] - g['revenue_median']
display(g.sort_values('mean_minus_median', ascending=False).head(15))

In [ ]:
one_sentence_insight('Mean- Median gap이 큰 상위 1~2개 그룹을 고르고, KPI를 어떻게 바꿀지 제안')

### 5.3 Outlier review: top 0.5% revenue rows
✅ 이상치는 제거 대상이 아니라 **설명 대상**입니다.

In [ ]:
q995 = df['revenue'].fillna(0).quantile(0.995)
out = df[df['revenue'].fillna(0) >= q995][['date','country','channel','segment','campaign','revenue','gross_margin','discount_rate']]
display(out.head(20))
print("outlier share of total revenue:", out['revenue'].sum() / df['revenue'].fillna(0).sum())

In [ ]:
one_sentence_insight('상위 0.5%가 전체 매출에서 차지하는 비중을 보고 KPI/리포팅 관점에서 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
